In [2]:
import os, re
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [35]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [4]:
train.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [36]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [37]:
train = train.groupby('target').sample(3000)

In [39]:
train_df, val_df = np.split(train.sample(frac = 1), [int(0.8 * len(train))])

In [41]:
def clean_text(dataframe):
    dataframe.text = dataframe.text.apply(lambda x: re.sub(r'http\S+', '', x))
    dataframe.text = dataframe.text.apply(lambda x: re.sub(r'\W+', ' ', x))
    dataframe.text = dataframe.text.apply(lambda x: re.sub(r'\d+', '', x))
    dataframe.text = dataframe.text.apply(lambda x: x.lower())
    return(dataframe)

In [42]:
cleaned_train = clean_text(train_df)
cleaned_val = clean_text(val_df)

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [50]:
vocab_size = 5000
embedding_dim = 100

tokenizer = Tokenizer(vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(cleaned_train.text)

train_sequence = tokenizer.texts_to_sequences(cleaned_train.text)
val_sequence = tokenizer.texts_to_sequences(cleaned_val.text)

maxlen = max([len(x) for x in train_sequence])
train_padded = pad_sequences(train_sequence, maxlen, padding = 'post', truncating = 'post')
val_padded = pad_sequences(val_sequence, maxlen, padding = 'post', truncating = 'post')

In [54]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100)))
model.add(tf.keras.layers.Dense(50, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(12, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

2023-07-15 23:24:24.010804: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-15 23:24:24.016892: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-15 23:24:24.022948: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [55]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)

In [56]:
model.fit(train_padded, 
          train_df.target,
          validation_data = (val_padded, val_df.target),
          epochs = 10)

Epoch 1/10


2023-07-15 23:24:32.710086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-15 23:24:32.713214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-15 23:24:32.716222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

150/150 [==============================] - ETA: 0s - loss: 0.6022 - accuracy: 0.6698

2023-07-15 23:24:45.949936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-15 23:24:45.953781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-15 23:24:45.955927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

150/150 [==============================] - 15s 59ms/step - loss: 0.6022 - accuracy: 0.6698 - val_loss: 0.4955 - val_accuracy: 0.7725
Epoch 2/10
150/150 [==============================] - 10s 68ms/step - loss: 0.3758 - accuracy: 0.8465 - val_loss: 0.5099 - val_accuracy: 0.7867
Epoch 3/10
150/150 [==============================] - 10s 64ms/step - loss: 0.2750 - accuracy: 0.8979 - val_loss: 0.5564 - val_accuracy: 0.7700
Epoch 4/10
150/150 [==============================] - 11s 72ms/step - loss: 0.2091 - accuracy: 0.9265 - val_loss: 0.6867 - val_accuracy: 0.7650
Epoch 5/10
150/150 [==============================] - 13s 85ms/step - loss: 0.1451 - accuracy: 0.9425 - val_loss: 0.9358 - val_accuracy: 0.7508
Epoch 6/10
150/150 [==============================] - 12s 79ms/step - loss: 0.1040 - accuracy: 0.9581 - val_loss: 1.1215 - val_accuracy: 0.7458
Epoch 7/10
150/150 [==============================] - 11s 74ms/step - loss: 0.1026 - accuracy: 0.9567 - val_loss: 1.2468 - val_accuracy: 0.7342
Epo

In [34]:
train_padded.shape

(5, 1)